In [1]:
#!/usr/bin/env python3.9
from torch import nn
from transformers import Seq2SeqTrainer,AutoTokenizer
from transformers import T5ForConditionalGeneration,EarlyStoppingCallback
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, AdamW, ProgressCallback
from tqdm import tqdm
import torch
import json
import pandas as pd
from datasets import  Dataset
from nltk.translate.bleu_score import sentence_bleu
import time
from datetime import timedelta
import wandb

In [2]:
# TODO: 
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: bollu (microsoft-research-incubation). Use `wandb login --relogin` to force relogin


True

In [3]:
model_name="./output_dir/checkpoint-500"
tokenizer_name = "kaiyuy/leandojo-lean4-sst-byt5-small"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
max_model_length = tokenizer.model_max_length #for CodeT5 it is 512
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

device: cuda


In [4]:
#updating the tokenizer's vocabulary file with End of Statement <EOS> Special Token:
end_of_proof_token='<EOS>' # TODO: change to tokenizer eof tken
special_tokens_dict = {'eos_token': end_of_proof_token}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(385, 1472)

In [5]:
# tokenizer(["foo", "bar"], max_length=tokenizer.model_max_length, truncation=True, padding="max_length")

In [9]:
save_directory='./model-finetuned-definition-prediction' #for saving the in the current working directory

# Opening JSON file
train = open('../../llm_benchmarking/fstar_insights/lemmas_with_premises/saikat_dataset_filtered_premises/initial_data/train.json')
valid = open('../../llm_benchmarking/fstar_insights/lemmas_with_premises/saikat_dataset_filtered_premises/initial_data/validate.json')
test = open('../../llm_benchmarking/fstar_insights/lemmas_with_premises/saikat_dataset_filtered_premises/initial_data/test.json')

train_data = json.load(train)
valid_data=json.load(valid)
test_data=json.load(test)

train_df = pd.DataFrame.from_dict(train_data, orient='columns')
valid_df = pd.DataFrame.from_dict(valid_data, orient='columns')
test_df = pd.DataFrame.from_dict(test_data, orient='columns')


train_df['input'] = train_df['name'] + '<SEP>' + train_df['type']
valid_df['input'] = valid_df['name'] + '<SEP>' + valid_df['type']
test_df['input'] = test_df['name'] + '<SEP>' + test_df['type']

#changing type of data to hugging face dataset for .map() and for Trainer class

train = train_df[['input', 'definition']]
valid = valid_df[['input', 'definition']]
test = test_df[['input', 'definition']]
train_dataset = Dataset.from_pandas(train)
valid_dataset = Dataset.from_pandas(valid)
test_dataset = Dataset.from_pandas(test)

print(train_dataset)
print(valid_dataset)
print(test_dataset)

def get_label_ids(target):
    max_length=tokenizer.model_max_length
    # to train model on End of statement token. Even When model truncates longer code, EOS remain to show model the end of the statement
    # Tokenize the target text without padding to get the tokens
    encoded_tokens = tokenizer.tokenize(target)
    # Check if the total number of tokens is greater than max_length
    if len(encoded_tokens) > max_length:
        # If yes, truncate the tokens while preserving the "<EOS>" at the end
        truncated_tokens = encoded_tokens[:max_length - 1] + [encoded_tokens[-1]]
        # Convert the truncated tokens back to input_ids
        input_ids = tokenizer.convert_tokens_to_ids(truncated_tokens)
    else:
        # If no truncation needed, keep the original tokens with padding
        input_ids = tokenizer(target,padding="max_length", truncation=True).input_ids
    return input_ids

def preprocess_data(data):  
    input = data['input']
    targets = data['definition']
    # inputs = [inp for inp in input + end_of_proof_token]
    model_inputs = tokenizer(input,
                             truncation=True, padding="max_length")
    # encode the summaries
    targets = [inp + end_of_proof_token for inp in targets]
    labels= [get_label_ids(t) for t in targets]
    # important: we need to replace the index of the padding tokens by -100
    # such that they are not taken into account by the CrossEntropyLoss
    labels_with_ignore_index = []
    for labels_example in labels:
        labels_example = [label if label != 0 else -100 for label in labels_example]
        labels_with_ignore_index.append(labels_example)
    # model_inputs["labels"] = labels_with_ignore_index
    model_inputs['decoder_input_ids'] = model_inputs['input_ids']
    return model_inputs

#maping the dataset into batches
train_dataset = train_dataset.map(preprocess_data, 
                                  batched=True,
                                  batch_size=8,
                                  remove_columns=["input", "definition"])
valid_dataset = valid_dataset.map(preprocess_data,
                                  batched=True,
                                  batch_size=8,
                                  remove_columns=["input", "definition"])
test_dataset = test_dataset.map(preprocess_data, batched=True,
                                batch_size=8,remove_columns=["input", "definition"])


def debug():
    print(train_dataset)
    for (i, record) in enumerate(train_dataset):
        print(f"--record {i}--")
        for k in record.keys():
            print(f"key: {k:20} | len: {len(record[k])}")
        record_len = len(record[list(record.keys())[0]])
        print(f"record len: {record_len}")
        record = { k : record[k][:5] for k in record.keys() }
        print(f"record: {record}")
        if i >= 5: break
debug()

Dataset({
    features: ['input', 'definition'],
    num_rows: 4325
})
Dataset({
    features: ['input', 'definition'],
    num_rows: 644
})
Dataset({
    features: ['input', 'definition'],
    num_rows: 644
})


Map:   0%|          | 0/4325 [00:00<?, ? examples/s]

/home/t-sibhat/.local/lib/python3.9/site-packages/transformers/models/byt5/tokenization_byt5.py:149: UserWarning: This sequence already has <EOS>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:70                                                                                   │
│                                                                                                  │
│   67 │   return model_inputs                                                                     │
│   68                                                                                             │
│   69 #maping the dataset into batches                                                            │
│ ❱ 70 train_dataset = train_dataset.map(preprocess_data,                                          │
│   71 │   │   │   │   │   │   │   │     batched=True,                                             │
│   72 │   │   │   │   │   │   │   │     batch_size=8,                                             │
│   73 │   │   │   │   │   │   │   │     remove_columns=["input", "definition"])                   │
│                                                                                                  │
│ /home/t-sibhat/.local/lib/python3.9/site-packages/datasets/arrow_dataset.py:592 in wrapper       │
│                                                                                                  │
│    589 │   │   else:                                                                             │
│    590 │   │   │   self: "Dataset" = kwargs.pop("self")                                          │
│    591 │   │   # apply actual function                                                           │
│ ❱  592 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)                │
│    593 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) else [ou  │
│    594 │   │   for dataset in datasets:                                                          │
│    595 │   │   │   # Remove task templates if a column mapping of the template is no longer val  │
│                                                                                                  │
│ /home/t-sibhat/.local/lib/python3.9/site-packages/datasets/arrow_dataset.py:557 in wrapper       │
│                                                                                                  │
│    554 │   │   │   "output_all_columns": self._output_all_columns,                               │
│    555 │   │   }                                                                                 │
│    556 │   │   # apply actual function                                                           │
│ ❱  557 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)                │
│    558 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) else [ou  │
│    559 │   │   # re-apply format to the output                                                   │
│    560 │   │   for dataset in datasets:                                                          │
│                                                                                                  │
│ /home/t-sibhat/.local/lib/python3.9/site-packages/datasets/arrow_dataset.py:3097 in map          │
│                                                                                                  │
│   3094 │   │   │   │   │   total=pbar_total,                                                     │
│   3095 │   │   │   │   │   desc=desc or "Map",                                                   │
│   3096 │   │   │   │   ) as pbar:                                                                │
│ ❱ 3097 │   │   │   │   │   for rank, done, content in Dataset._map_single(**dataset_kwargs):     │
│   3098 │   │   │   │   │   │   if done:                                                          │
│   3099 │   │   │   │   │   │   │   shards_done += 1                                              │
│   3100 │   │   │   │   │   │   │   logger.debug(f"Finished 

In [7]:
#Trainer Class Arguments:
args = Seq2SeqTrainingArguments(
    output_dir="model-finetuned-definition-prediction",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    learning_rate=5e-5,
    do_eval=True,
    do_train=True,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=2,
    eval_accumulation_steps=2,
    num_train_epochs=15,
    warmup_steps=500,
    weight_decay=0.01,
    eval_steps=100,             
    save_steps=500,  
    load_best_model_at_end=True,
)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer) # no masked language modelling


trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)
print(f"trainer: {trainer}")

trainer: <transformers.trainer_seq2seq.Seq2SeqTrainer object at 0x7fe245062ca0>


In [8]:
run = wandb.init(
    # Set the project where this run will be logged
    project="lean3-codet5-fstar-finetune-definition-prediction",
    # Track hyperparameters and run metadata
    config={})
trainer.train()
model.save_pretrained(save_directory)
trainer.save_model(save_directory)
trainer.save_state()

/home/t-sibhat/.local/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│    3 │   project="lean3-codet5-fstar-finetune-definition-prediction",                            │
│    4 │   # Track hyperparameters and run metadata                                                │
│    5 │   config={})                                                                              │
│ ❱  6 trainer.train()                                                                             │
│    7 model.save_pretrained(save_directory)                                                       │
│    8 trainer.save_model(save_directory)                                                          │
│    9 trainer.save_state()                                                                        │
│                                                                                                  │
│ /home/t-sibhat/.local/lib/python3.9/site-packages/transformers/trainer.py:1645 in train          │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /home/t-sibhat/.local/lib/python3.9/site-packages/transformers/trainer.py:1938 in                │
│ _inner_training_loop                                                                             │
│                                                                                                  │
│   1935 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1936 │   │   │   │                                                                             │
│   1937 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1938 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1939 │   │   │   │                                                                             │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /home/t-sibhat/.local/lib/python3.9/site-packages/transformers/trainer.py:2759 in training_step  │
│                                                                                                  │
│   2756 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2757 │   │                                                                                     │
│   2758 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2759 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2760 │   │                                                                                     │
│   2761 │   │   if self.args.n_gpu > 1:                     

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(save_directory).to(device) #loading the fine-tuned model

#Testing the Model:

def get_test_results(test_dataset):
        num_beams = 15 #can chnage upto 50
        temperature = 1.0
        predictions = []
        actuals = []
        x=0
        with torch.no_grad():
            for data in tqdm(test_dataset):
                bleu_scores = []
                test=data['input']
                label_=data['definition']
                input_ids = tokenizer(test, return_tensors='pt').input_ids.to(device)
                output_sequences = model.generate(input_ids, 
                                                  num_beams=num_beams,
                                                  temperature=temperature,
                                                  num_return_sequences=15,
                                                  max_length=max_model_length)

                candidate_definitions = [tokenizer.decode(seq, skip_special_tokens=True,clean_up_tokenization_spaces=True)for seq in output_sequences ]
                for candidate_sentence in candidate_definitions: #calculating sentence BLEU between label and each of the generated candiadte
                            bleu_score = sentence_bleu([candidate_sentence.split()],label_.split())
                            bleu_scores.append(bleu_score) 
                            
                # Find the index of the candidate with the highest BLEU score
                print(bleu_scores)
                highest_bleu_index = bleu_scores.index(max(bleu_scores))
                print(highest_bleu_index)
                # Get the candidate sentence with the highest BLEU score 
                best_pred = candidate_definitions[highest_bleu_index]
                predictions.append(best_pred)
                actuals.append(label_)
                print(len(predictions))
                print(len(actuals))
                x=x+1

        return predictions,actuals

#testing the trained model on test dataset
start_time = time.time()
print(f"run started at {start_time}")
predictions,actuals=get_test_results(test_dataset) #get model's generated definitions and actual lemma definitions
elapsed_time_secs = time.time() - start_time
print("Total Inference Time: ",round(elapsed_time_secs),'seconds')


#Calculate Sentence BLEU Score and exact Match

def calculate_sentence_bleu(candidate, reference):
    bleu_score = sentence_bleu([candidate.split()], reference.split())
    return bleu_score

def calculate_average_bleu(predictions, actuals):
    assert len(predictions) == len(actuals), "Both predictions and actuals must have the same number of definitions."

    total_sentence_bleu=[]
    num_sentences = len(predictions)

    for i in tqdm(range(num_sentences)):
        sentence_bleu = calculate_sentence_bleu(predictions[i],actuals[i])        
        total_sentence_bleu.append(sentence_bleu)

    
    average_sentence_bleu = sum(total_sentence_bleu) / num_sentences
    return average_sentence_bleu,total_sentence_bleu


average_sentence_bleu,total_sentence_bleu= calculate_average_bleu(predictions, actuals)

print("Average Sentence BLEU:", round(average_sentence_bleu*100), '%')
print("Sentence BLEU for each lemma:    ",total_sentence_bleu)


predictions_tokens = [prediction.split() for prediction in predictions]
labels_tokens = [label.split() for label in actuals]

exact_match_count = 0
for prediction_tokens, label_tokens in zip(predictions_tokens, labels_tokens):
    if prediction_tokens == label_tokens:
        exact_match_count += 1
# Calculate the exact match score
exact_match_score = exact_match_count / len(predictions_tokens)

print(f"Exact Match Score: {exact_match_score:.2f}")

print('Generated lemmas: ',predictions)
print('Actual lemmas: ',actuals)
trainer.state.log_history